In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import scipy.misc  # for image resizing

#import scipy.io.wavfile

# pip install soundfile
import soundfile

from IPython.display import Audio as audio_playback_widget

In [ ]:
#f = './data/raw-from-phone.wav'  # un-normalized
f = './data/num_phone_en-UK_m_Martin14.wav'  # has been normed

In [ ]:
# Read in the original file
samples, sample_rate = soundfile.read(f)

samples = samples / np.max(samples)  # Norm the signal

def show_waveform(sound):
    n_samples = sound.shape[0]

    plt.figure(figsize=(12,2))
    plt.plot(np.arange(0.0, n_samples)/sample_rate, sound)
    plt.xticks( np.arange(0.0, n_samples/sample_rate, 0.5), rotation=90 )

    plt.grid(True)

    plt.show()

show_waveform(samples)
audio_playback_widget(f)

In [ ]:
n_fft = 512

fft_step   = 0.010 # 10ms
fft_window = 0.025 # 25ms

In [ ]:
import librosa

#hop_length=int(fft_step*sample_rate)   # number audio of frames between STFT columns
#win_length=int(fft_window*sample_rate) # number audio of frames between STFT columns

win_length=None # defaults to n_fft
hop_length=None # defaults to win_length/4

spectrum_complex = librosa.stft(samples, n_fft=n_fft, 
                   hop_length=hop_length, win_length=win_length,
                   window='hann', center=True, 
                   dtype=np.complex64) # This has real and imaginary parts each as float32

spectrum_complex.shape

In [ ]:
samples_defft = librosa.istft(spectrum_complex, hop_length=hop_length, win_length=win_length)

In [ ]:
def quick_view_and_play(stub, samples_here):
    f = './tmp/%s.wav' % (stub,)
    soundfile.write(f, samples_here/np.max(samples_here), samplerate=sample_rate)
    show_waveform(samples_here)
    return audio_playback_widget(f)

quick_view_and_play('defft', samples_defft)

In [ ]:
spectrum_real = np.absolute(spectrum_complex)
# effectively, all phase information==0

samples_re_defft = librosa.istft(spectrum_real, hop_length=hop_length, win_length=win_length)
quick_view_and_play('re-defft', samples_re_defft)

In [ ]:
def real_spectrum_to_samples( spectrum_re ):
    phases = 2.0 * np.pi * np.random.random_sample(spectrum_re.shape) - np.pi
    #phases = np.zeros_like( spectrum_re )

    for i in range(20):
        spectrum_complex_guess = spectrum_re * np.exp(1.j*phases)

        samples_reim = librosa.istft(spectrum_complex_guess, 
                                     hop_length=hop_length, win_length=win_length,
                                     window='hann', center=True, 
                                    )

        re_calc_fft = librosa.stft(samples_reim, n_fft=n_fft, 
                       hop_length=hop_length, win_length=win_length,
                       window='hann', center=True, 
                       dtype=np.complex64)

        phases_next = np.angle( re_calc_fft )  # What are the phases just reported?  Next iteration, use these
        #phases = (phases+phases_next)/2.0
        
        phases_diff = (phases_next - phases + np.pi) % (2 * np.pi ) - np.pi
        
        phases_clipped = np.clip( phases_diff, a_min=-np.pi/8.0, a_max=+np.pi/8.0)
        phases = phases + phases_clipped
        
        print( [ '%+.4f' % p for p in phases_clipped[30:40, int(5.3/fft_step)] ] )
        #print( np.abs(phases_diff).mean() )
        
    return samples_reim
    
samples_reim_defft = real_spectrum_to_samples( spectrum_real )

quick_view_and_play('reim-defft', samples_reim_defft)

In [ ]:
import python_speech_features
n_mel_freq_components = 64

# create_mel_filters
mel_inversion_filter = python_speech_features.get_filterbanks(nfft=n_fft, samplerate=sample_rate,
                                        nfilt=n_mel_freq_components,
                                        lowfreq = 300.0, highfreq = 8000.0)
                                        #lowfreq = 0, highfreq = None)

mel_filter = mel_inversion_filter.T / mel_inversion_filter.sum(axis=1)
mel_filter.shape

In [ ]:
def make_mel(spectrogram, mel_filter, shorten_factor=1.0):
    #mel_spec = np.transpose(mel_filter).dot( np.transpose(spectrogram) )
    #spectrogram = np.square(spectrogram)  # Convert to 'power'
    mel_spec = np.transpose(spectrogram).dot( mel_filter )
    #mel_spec = scipy.ndimage.zoom(mel_spec.astype('float32'), [1, 1./shorten_factor]).astype('float16')
    mel_spec = scipy.ndimage.zoom(mel_spec.astype('float32'), [1./shorten_factor, 1]).astype('float16')
    #mel_spec = mel_spec[:,1:-1] # a little hacky but seemingly needed for clipping 
    return mel_spec

def mel_to_spectrogram(mel_spec, mel_inversion_filter, shorten_factor=1.0, spec_thresh=0.0):
    """
    takes in an mel spectrogram and returns a normal spectrogram for inversion 
    """
    mel_spec = (mel_spec+spec_thresh)
    uncompressed_spec = scipy.ndimage.zoom(mel_spec.astype('float32'), [shorten_factor,1]).astype('float16')
    #uncompressed_spec = np.transpose(np.transpose(mel_spec).dot(mel_inversion_filter))
    uncompressed_spec = np.transpose( uncompressed_spec.dot(mel_inversion_filter) )
    #uncompressed_spec = uncompressed_spec -4
    #uncompressed_spec = np.sqrt(uncompressed_spec)  # Convert from 'power'
    return uncompressed_spec

In [ ]:
shorten_factor=2.0

#mel_spec = make_mel(spectrum_real, mel_filter, shorten_factor=shorten_factor)
mel_spec = make_mel(spectrum_real/np.max(spectrum_real), mel_filter, shorten_factor=shorten_factor)

#spectrum_real
mel_spec.shape

In [ ]:
# plot the compressed spec
fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(20,4))

cax = ax.matshow(np.log(mel_spec.T), interpolation='nearest', aspect='auto', cmap=plt.cm.afmhot, origin='lower')
fig.colorbar(cax)
plt.title('mel Spectrogram')
plt.show()

In [ ]:
spectrogram_from_mel = mel_to_spectrogram(mel_spec, mel_inversion_filter,
                                          shorten_factor=shorten_factor)

In [ ]:
# plot the recovered spec
fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(20,4))

cax = ax.matshow(spectrogram_from_mel, interpolation='nearest', aspect='auto', cmap=plt.cm.afmhot, origin='lower')
fig.colorbar(cax)
plt.title('mel Spectrogram')
plt.show()

In [ ]:
samples_via_mel = real_spectrum_to_samples( spectrogram_from_mel )

quick_view_and_play('via-mel', samples_via_mel)